# Notebook 2: mesh processing

In this notebook, we will learn some basics about producing meshes from segmentations. 

## Load the data and visualize
We will load a segmentation from the path below. The segmentation is generated from a simulation of a 4 cell embryo ([SimuCell3D](https://www.nature.com/articles/s43588-024-00620-9)).

In the cell below launch the napari viewer and add the segmentation as a Labels layer. Switch to 3D mode and view the data.

<img src="resources/napari_embryo_segmentation.png" alt="napari surface layer">

In [2]:
from skimage.io import imread

segmentation_path = "../data/part_2/4_cells_simulation_0.tif"
segmentation = imread(segmentation_path) 

In [3]:
import napari

viewer = napari.Viewer()
viewer.add_labels(segmentation)


<Labels layer 'segmentation' at 0x112a379d0>

## Converting a segmentation to a mesh

First, we will extract one cell from the segmentation to turn it into a mesh. Start by inspecting the data. What values are present? Hint: you can use the numpy unique function to find unique values in the array.

In [4]:
import numpy as np

label_values = np.unique(segmentation)
print(f"Label values: {label_values}")

Label values: [0 1 2 3 4]


How many cells are there? Remember that 0 has a special meaning in label images.

Select one of the cell labels and create a binary mask with only that cell. Add it to the viewer as a new labels layer.

<img src="resources/extracted_cell.png" alt="napari surface layer">

In [5]:
selected_label_value = 1

# make the mask that is True for voxels
# with the value of the selected label value
cell_mask = segmentation == selected_label_value


# add it to the viewer as a labels image (convert to uint datatype)
viewer.add_labels(cell_mask.astype(np.uint16))

<Labels layer 'Labels' at 0x176c0d650>

Now we will convert the cell mask into a mesh using the [marching cubes algorithm](https://en.wikipedia.org/wiki/Marching_cubes). This converts the surface of the cell mask into a triangulated surface.

Hints:
- see the [skimage marching cubes docs](https://scikit-image.org/docs/0.25.x/api/skimage.measure.html#skimage.measure.marching_cubes)
- see the [skimage marching cubes example](https://scikit-image.org/docs/0.25.x/auto_examples/edges/plot_marching_cubes.html)
- We only need the vertices and faces
- Set the new surface layer to be the only layer that is visible. Overlaying the segmentation and mesh can be confusing as they overlap.
- Set the surface layer wireframe to be visible (as before) to visualize the triangles in the mesh.

In [6]:
from skimage.measure import marching_cubes

vertices, faces, _, _ = marching_cubes(cell_mask, 0)

cell_mesh_layer = viewer.add_surface((vertices, faces))
cell_mesh_layer.wireframe.visible = True

You should now see the mesh of cell. Be sure to set the surface layer wireframe to be visible (as before) to visualize the triangles in the mesh. Notice that the surface is quite bumpy. This is because the marching cubes algorithm uses the voxel surface to construct the surface, which gives this "blocky" appearance. In the notebook, we will try the surface reconstruction method introduced in Foambryo, which yields accurate and smooth surfaces

<img src="resources/cell_mesh.png" alt="napari surface layer">

## Surface area

We can use the trimesh library measure the surface area of the cell and then visualize it in napari.

First we have to create the Trimesh object.


In [7]:
import trimesh

mesh = trimesh.Trimesh(
    vertices=vertices,
    faces=faces
)

Then we apply measure the surface area and add the mesh to the viewer.

In [8]:
surface_area = mesh.area

# make the vertex values equal to the surface area
n_vertices = vertices.shape[0]
vertex_values = surface_area * np.ones((n_vertices,))

viewer.add_surface((vertices, faces, vertex_values))

<Surface layer 'Surface [1]' at 0x17eb0dc90>

Note that all vertices have the same color because they have the same value. Try the bonus exercise below to visualize with different colors.


<img src="resources/cell_mesh_color.png" alt="napari surface layer">

## Bonus Exercise

Write a function that measures the surface area of each cell in the segmentation. Add each cell mesh as a separate layer to the viewer and color by the cells by their surface area. Note that have to set the [contrast limits](https://napari.org/stable/api/napari.layers.Surface.html) for each layer to the same range to visualize the color map correctly.